In [165]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Electrode coordinates inter-individual variability.

import numpy as np
import os
import glob

## Parameters and variables: 
coords_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\electrode_coords\\'

# List of electrodes coordiantes (already in the MNI space)
wp1a_elect_anod = np.load(os.path.join(coords_folder, 'wp1a_anod_mni_coords.npy'), allow_pickle = True)
wp1a_elect_anod = np.asarray(list(dict(enumerate(wp1a_elect_anod.flatten(), 1))[1].values()))

wp2a_elect_anod = np.load(os.path.join(coords_folder, 'wp2a_anod_mni_coords.npy'), allow_pickle = True)
wp2a_elect_anod = np.asarray(list(dict(enumerate(wp2a_elect_anod.flatten(), 1))[1].values()))

wp1b_elect_cath = np.load(os.path.join(coords_folder, 'wp1b_cath_mni_coords.npy'), allow_pickle = True)
wp1b_elect_cath = np.asarray(list(dict(enumerate(wp1b_elect_cath.flatten(), 1))[1].values()))

In [167]:
wp1a_mean_point = [np.mean(wp1a_elect_anod[:,0,0]), np.mean(wp1a_elect_anod[:,0,1]),
                   np.mean(wp1a_elect_anod[:,0,2])] # x y z mean for wp1a anod
wp1a_std_point = [np.std(wp1a_elect_anod[:,0,0]), np.std(wp1a_elect_anod[:,0,1]),
                   np.std(wp1a_elect_anod[:,0,2])]

wp1a_std_magnitude = np.sqrt(wp1a_std_point[0]**2 + wp1a_std_point[1]**2 + wp1a_std_point[2]**2)

print('average elect position')
print(wp1a_mean_point)
print('average distance from mean point (in mm)')
print(wp1a_std_magnitude)

dists = []
for point in wp1a_elect_anod:
    dist = np.linalg.norm(wp1a_mean_point-point)
    dists.append(dist)
print(np.mean(dists))
print(np.std(dists))

average elect position
[-47.67993957887996, -20.348977614532817, 70.32529149543156]
average distance from mean point (in mm)
23.015224847051435
19.162455611059638
12.747582896945714


In [168]:
wp1b_mean_point = [np.mean(wp1b_elect_cath[:,0,0]), np.mean(wp1b_elect_cath[:,0,1]),
                   np.mean(wp1b_elect_cath[:,0,2])] # x y z mean for wp1a anod
wp1b_std_point = [np.std(wp1b_elect_cath[:,0,0]), np.std(wp1b_elect_cath[:,0,1]),
                   np.std(wp1b_elect_cath[:,0,2])]

wp1b_std_magnitude = np.sqrt(wp1b_std_point[0]**2 + wp1b_std_point[1]**2 + wp1b_std_point[2]**2)

print('average elect position')
print(wp1b_mean_point)

print('average distance from mean point (in mm)')
print(wp1b_std_magnitude)


# this should give very similar results, but we get the std of the distance as well
dists = []
for point in wp1b_elect_cath:
    dist = np.linalg.norm(wp1b_mean_point-point)
    dists.append(dist)
print(np.mean(dists))
print(np.std(dists))

average elect position
[40.92291050865536, -91.4834630773181, -19.570283168838138]
average distance from mean point (in mm)
21.615473698122862
19.386361777338966
9.560213398892941


In [169]:
wp2a_mean_point = [np.mean(wp2a_elect_anod[:,0,0]), np.mean(wp2a_elect_anod[:,0,1]),
                   np.mean(wp2a_elect_anod[:,0,2])] # x y z mean for wp1a anod
wp2a_std_point = [np.std(wp2a_elect_anod[:,0,0]), np.std(wp2a_elect_anod[:,0,1]),
                   np.std(wp2a_elect_anod[:,0,2])]

wp2a_std_magnitude = np.sqrt(wp2a_std_point[0]**2 + wp2a_std_point[1]**2 + wp2a_std_point[2]**2)

print('average elect position')
print(wp2a_mean_point)
print('average distance from mean point (in mm)')
print(wp2a_std_magnitude)

dists = []
for point in wp2a_elect_anod:
    dist = np.linalg.norm(wp2a_mean_point-point)
    dists.append(dist)
print(np.mean(dists))
print(np.std(dists))


average elect position
[-57.58827231722799, -19.233482379127633, 48.25523163784634]
average distance from mean point (in mm)
19.67566752836689
16.45523234608223
10.786900440964514


In [200]:
# Comparison between wp1a and wp2a in terms of electrode distance from M1 centroid
from nilearn import image
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' # Project folder

## Loading AAL3 atlas and extracting M1 / Thalamus ROIs (regions of interest)
# AAL3 atlas paper: https://www.oxcns.org/papers/607%20Rolls%20Huang%20Lin%20Feng%20Joliot%202020%20AAL3.pdf 
AAl3_path = os.path.join(main_folder, 'rois', 'AAL3v1_1mm.nii')
AAL3_atlas = image.load_img(AAl3_path)

## Creating M1, Th and cerebellar masks from the AAL3 atlas. Load MNI template.
# AAL3 index for left M1 = 1
m1 = image.math_img("np.where(img == 1, 1, 0)", img = AAL3_atlas) 
m1_narray = m1.get_fdata()

def matrix_to_mni(matrix_coord, T):
    # From matrix space to MNI space
        second_arg = np.array([matrix_coord[0],matrix_coord[1],matrix_coord[2], 1])
        second_arg = np.reshape(second_arg,[-1,1])
        mni_coord = np.dot(T,second_arg)

        return np.reshape(mni_coord[0:3],[1,-1])


# centroid of M1 
from scipy import ndimage

m1_centre_of_mass = ndimage.measurements.center_of_mass(m1_narray)
m1_centre_of_mass = matrix_to_mni(m1_centre_of_mass,AAL3_atlas.affine)       

dist_wp1a = []
for point in wp1a_elect_anod:
    dist = np.linalg.norm(point-m1_centre_of_mass)
    dist_wp1a.append(dist)

print(np.mean(dist_wp1a))
print(np.std(dist_wp1a))

dist_wp2a = []
for point in wp2a_elect_anod:
    dist = np.linalg.norm(point-m1_centre_of_mass)
    dist_wp2a.append(dist)
print(np.mean(dist_wp2a))
print(np.std(dist_wp2a))

In [202]:
dist_wp2a

[39.16213305179278,
 19.53475585151319,
 20.987503173105992,
 34.74753807206986,
 38.6401930834467,
 27.027847928100865,
 30.264443964112754,
 27.57377416418355,
 38.936459283376045,
 36.66133739234481,
 29.517891026896034,
 19.429911712494167,
 27.869352473385433,
 29.711514906033806,
 24.92601775832279,
 28.90841382757443,
 28.193915083358583,
 36.809432259472686,
 25.9308859645877,
 18.752016989016713,
 22.4166808092647,
 35.95604842109359]